### Import external dependencies.

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import zipfile

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision import transforms

### Import repo from repo github.
For more info about it, please refer to https://github.com/juanprida/image-classification

In [5]:
!cp -r ../input/image-classification-folder/* ./
from image_classification.cat_dog_dataset import CatDogDataset
from image_classification.alexnet import AlexNet
from image_classification.engine import Engine

### Get data and define constants.
We are working with Kaggle notebooks, you should add https://www.kaggle.com/competitions/dogs-vs-cats dataset.

In [6]:
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip',"r") as train:
    train.extractall()

train_dir = './train'
train_files = os.listdir(train_dir)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Execute model.

In [25]:
n_epochs = 20
learning_rate = 1e-3
weight_decay = 5e-4
batch_size = 1028

data_transform = transforms.Compose([
    transforms.Resize(227),
    transforms.ColorJitter(),
    transforms.RandomCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(227),
    transforms.ToTensor()
])

train_files, val_files = train_test_split(train_files)

train_data = CatDogDataset(train_files, train_dir, data_transform)
val_data = CatDogDataset(val_files, train_dir, data_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

model = AlexNet()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

opt = Engine(model=model, loss_fn=loss_fn, optimizer=optimizer, device=device)
opt.train(train_loader, val_loader, n_epochs)
opt.plot_losses()

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[1/20] Training loss: 0.8338	 Validation loss: 0.6920	 Validation accuracy: 0.5192
[2/20] Training loss: 0.7058	 Validation loss: 0.7005	 Validation accuracy: 0.4792
[3/20] Training loss: 0.6938	 Validation loss: 0.7042	 Validation accuracy: 0.4760
[4/20] Training loss: 0.6763	 Validation loss: 0.7397	 Validation accuracy: 0.4776
[5/20] Training loss: 0.6645	 Validation loss: 0.7647	 Validation accuracy: 0.4776
[6/20] Training loss: 0.6528	 Validation loss: 0.7935	 Validation accuracy: 0.4776


KeyboardInterrupt: 